# 训练数据处理

* 处理40%以上空缺的列
* 根据先验知识除去不需要的列
* 添加habitable列
* 将分类种类大于10类的列删除
* 填补空缺项（分类列填missing，数值列填nan）
* 剩余的分类列做one-hot编码
* 对数据进行标准化处理

In [27]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib
import pickle


hpl_data = pd.read_csv('./data/phl_exoplanet_catalog.csv')
exoplanet_data = pd.read_csv('./data/exoplanets.csv')
imputed_data = pd.read_csv('./data/ImputedNumericCols.csv')

# 获取hpl_data中的所有可宜居星球的名字
hpl_data = hpl_data.drop('P_STATUS', axis=1)
habitable_data = hpl_data[hpl_data.P_HABITABLE != 0]
habitable_planets_names = habitable_data.P_NAME.values

exoplanet_data["habitable"]  = exoplanet_data['pl_name'].isin(habitable_planets_names)

# 合并imputed_data和exoplanet_data
for field in imputed_data.columns:
    if field not in exoplanet_data.columns:
        pass
    else:
        exoplanet_data[field] = imputed_data[field]

# 去除不需要的列        
remove=['pl_controvflag', 'pl_letter', 'pl_discmethod', 'pl_nnotes', 'ra_str', 'dec_str', 'rowupdate', 'pl_tsystemref', 
        'pl_def_reflink', 'pl_disc', 'pl_disc_reflink', 'pl_locale', 'pl_facility', 'pl_telescope', 
        'pl_instrument', 'pl_status', 'pl_pelink', 'st_nts', 'st_nplc', 'st_nglc', 'st_nrvc', 'st_naxa', 
        'st_nimg', 'st_nspec', 'st_photn', 'st_colorn', 'pl_hostname', 'ra_str', 'dec_str', 'rowupdate', 
        'pl_def_reflink', 'pl_disc_reflink', 'pl_pelink', 'pl_edelink', 'pl_publ_date', 'hd_name', 
        'hip_name', 'st_spstr', 'swasp_id', 'pl_ttvflag', 'pl_kepflag', 'pl_k2flag', 'st_optband', 'st_metratio',
        'pl_imgflag', 'pl_astflag', 'pl_omflag', 'pl_cbflag', 'st_tefflim', 'pl_tranmidlim', 'pl_ratrorlim',
        'pl_ratrorlim', 'pl_mnum', 'st_plxlim', 'gaia_plxlim', 'gaia_distlim', 'st_pmralim', 'st_pmdeclim',
        'st_pmlim', 'gaia_pmralim', 'gaia_pmdeclim', 'gaia_pmlim', 'st_logglim', 'st_metfelim', 'st_distlim',
        'st_optmaglim', 'gaia_gmaglim', 'st_masslim', 'st_radlim', 'pl_trandurlim']
exoplanet_data = exoplanet_data.drop(remove, axis=1)           


# 去除超过15%缺失值的列
def moreThan15Missing(col):
    numMissing = len(exoplanet_data[exoplanet_data[col].isnull()])
    if numMissing / len(exoplanet_data) > 0.15:
        return 1
    return 0
       
SignificantMissingData = [x for x in exoplanet_data.columns if moreThan15Missing(x)]
exoplanet_data = exoplanet_data.drop(SignificantMissingData, axis=1)

print("exos after removing columns with more than 15% missing data",
       exoplanet_data.shape) 




exos after removing columns with more than 15% missing data (3970, 98)


In [28]:
# 获取分类列
Cat = [x for x in exoplanet_data.columns if x not in exoplanet_data._get_numeric_data().columns]
for field in Cat:
    if(len(exoplanet_data[field].unique()) > 10):
        exoplanet_data = exoplanet_data.drop(field, axis=1)
Cat = [x for x in exoplanet_data.columns if x not in exoplanet_data._get_numeric_data().columns]

print(exoplanet_data.shape)

# 填充缺失值
for field in exoplanet_data.columns:
    if(field in Cat):
        exoplanet_data[field] = pd.Categorical(exoplanet_data[field])
        exoplanet_data[field] = exoplanet_data[field].cat.add_categories("Missing").fillna("Missing")
    else: 
        exoplanet_data[field] = exoplanet_data[field].fillna(np.nan)
        
# 将rowid设置为索引
exoplanet_data = exoplanet_data.set_index("rowid")

# print(exoplanet_data.columns)

# 对可宜居星球和不可宜居星球做分别做处理
habitable = exoplanet_data[exoplanet_data.habitable == True]
nonHabitable = exoplanet_data[~exoplanet_data.habitable == True]

# 用中位数填补空缺值
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median') 
# habitable[habitable._get_numeric_data().columns] = imputer.fit_transform(habitable[habitable._get_numeric_data().columns])
# nonHabitable[nonHabitable._get_numeric_data().columns] = imputer.fit_transform(nonHabitable[nonHabitable._get_numeric_data().columns])
# imputer.fit(exoplanet_data[exoplanet_data._get_numeric_data().columns])
exoplanet_data[exoplanet_data._get_numeric_data().columns] = imputer.fit_transform(exoplanet_data[exoplanet_data._get_numeric_data().columns])

# 保存模型
joblib.dump(imputer, './model/imputer.joblib') 

# Join the two datasets
# tempExos = pd.concat([habitable,nonHabitable])

# for field in tempExos.columns:
#     if(field in exoplanet_data.columns):
#         exoplanet_data[field] = tempExos[field]
#     else:
#         print(field)
        
        
#Scale the data so that it has unit variance
NumericCols=[]
for n in exoplanet_data._get_numeric_data().columns:
	#Dont scale binary columns(ie those that only have 1 or 0)
    if not (list(exoplanet_data[n].unique()) == [0,1]):
        NumericCols.append(n)
scaler = StandardScaler()
exoplanet_data[NumericCols] = scaler.fit_transform(exoplanet_data[NumericCols])

with open('./model/NumericCols.pkl', 'wb') as f:
    pickle.dump(NumericCols, f)
# 保存模型
joblib.dump(scaler, './model/scaler.joblib')

print(exoplanet_data.shape)

#One hot encode categorical columns
preprocessed = pd.get_dummies(exoplanet_data)

print(preprocessed.shape)

preprocessed.to_csv('./preprocessed22222222.csv')

(3970, 97)
(3970, 96)
(3970, 96)
